<h3> Titanic_predict_tf_LR_DNN </h3>

In [84]:
import numpy as np
import pandas as pd
# import sys
import re
import time
import shutil
import itertools
import matplotlib.pyplot as plt
import tensorflow as tf
import xgboost
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.python.client import device_lib
from sklearn.metrics import accuracy_score
model_path = '/home/lambert/Git/Titanic/Titanic_model/'

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [3]:
def get_title(name):
    if pd.isnull(name):
        return 'Null'
    title_search = re.search('([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1).lower()
    else:
        return 'None'

In [4]:
titles = {'mr':1, 'mrs':2, 'mme':2, 'ms':3, 'miss':3, 'mlle':3, 'don':4, 'sir':4, 'jonkheer':4,
          'major':4, 'col':4, 'dr':4, 'master':4, 'capt':4, 'dona':5, 'lady':5, 'countess':5,
         'rev':6}
df_train['Title'] = df_train['Name'].apply(lambda name: titles.get(get_title(name)))
df_train['Honor'] = df_train['Title'].apply(lambda title: 1 if title ==4 or title == 5 else 0)
df_test['Title'] = df_test['Name'].apply(lambda name: titles.get(get_title(name)))
df_test['Honor'] = df_test['Title'].apply(lambda title: 1 if title ==4 or title == 5 else 0)

In [5]:
# 1. Data clearning
# df_train.Age.fillna(fill_df_Age, inplace=True)
# df_test.Age.fillna(fill_df_Age, inplace=True)
for i in range(1,7):
    a = df_train[(df_train.Age.isnull()) & (df_train.Title==i)]
    df_train.Age.iloc[a.index] = df_train.Age[df_train.Title==i].median()
    b = df_test[(df_test.Age.isnull()) & (df_test.Title==i)]
    df_test.Age.iloc[b.index] = df_train.Age[df_train.Title==i].median()

# Cabin - fill nan with 'fill' and select first cabin
df_train.Cabin.fillna('fill',inplace=True)
df_train.Cabin = df_train.Cabin.map(lambda x:x.split(' ')[0])
df_test.Cabin.fillna('fill', inplace=True)
df_test.Cabin = df_test.Cabin.map(lambda x:x.split(' ')[0])

# Sex - male: 1, female: 1
df_train.Sex.replace({'male':1, 'female':0}, inplace=True)
df_test.Sex.replace({'male':1, 'female':0}, inplace=True)

df_train = df_train.dropna(axis=0, how='any')
df_test.Fare.fillna(df_test.Fare.mean(), inplace=True)

# Create a 'Deceased' column for sconed class
df_train['Deceased'] = df_train.Survived.apply(lambda s: int(not s))

/Users/lambert/virtualenvs/tensorflow/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
# 2. Select the features
sFeatures = ['Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',             
 'Cabin',             
 'Embarked',
 'Title',
 'Honor']
LABEL = ['Survived', 'Deceased']

In [7]:
# 3. Label Encoder
# X_test has elements that X doesn't have
# create a Cabin_labels cover X and X_test
X_Cabin_unique = df_train['Cabin'].unique()
X_lack = df_test.Cabin[df_test['Cabin'].isin(X_Cabin_unique)==0].values
Cabin_labels = np.append(X_Cabin_unique, X_lack)
# print(X_Cabin_unique.shape, X_Cabin_unique)
# print(Cabin_labels.shape, Cabin_labels)

le_Cabin = LabelEncoder().fit(Cabin_labels)
le_Embarked = LabelEncoder().fit(np.array(df_train['Embarked'].tolist()))

df_train.loc[:,'Cabin'] = le_Cabin.transform(df_train['Cabin'])
df_train.loc[:,'Embarked'] = le_Embarked.transform(df_train['Embarked'])
df_test.loc[:,'Cabin'] = le_Cabin.transform(df_test['Cabin'])
df_test.loc[:,'Embarked'] = le_Embarked.transform(df_test['Embarked'])

In [8]:
# Do the feature engineer in DF, keep train and label index consistance
# Split Validation with the selected features
# X_train, X_val, y_train, y_val = train_test_split(X, y)
X_train, X_valid, y_train, y_valid = train_test_split(df_train[sFeatures], df_train['Survived'])
X_test = df_test[sFeatures]

In [9]:
# No Validation
# X_train = df_train[sFeatures]
# y_train = df_train['Survived']

### Feature columns

In [10]:
def make_feature_cols(X):
  return [tf.feature_column.numeric_column(k) for k in X.columns]

### Input function

In [11]:
# for training and test input function
def make_input_fn(X, y, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = X,
    y = y,
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000,
    num_threads = 1
  )

In [12]:
def make_prediction_input_fn(X, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = X,
    batch_size = 1,
    num_epochs = num_epochs,
    shuffle = False,
    queue_capacity = 1000,
    num_threads = 1
  )

<h3> Linear Regression with tf.Estimator framework </h3>

In [13]:
X_train.shape

(666, 10)

In [69]:
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR_LR = 'Titanic_trained_LC'
shutil.rmtree(OUTDIR_LR, ignore_errors = True) # start fresh each time

model_LR = tf.estimator.LinearClassifier(
    feature_columns = make_feature_cols(X_train), model_dir = OUTDIR_LR)

# model_LR = tf.estimator.LinearRegressor(
#       feature_columns = make_feature_cols(X_train), model_dir = OUTDIR_LR)

model_LR.train(input_fn = make_input_fn(X_train, y_train, num_epochs = 2000))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'Titanic_trained_LC', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1198b64a8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into Titanic_trained_LC/model.ckpt.
INFO:tensorflow:loss = 88.722855, step = 1
INFO:tensor

INFO:tensorflow:loss = 56.595444, step = 7401 (0.317 sec)
INFO:tensorflow:global_step/sec: 321.869
INFO:tensorflow:loss = 54.80037, step = 7501 (0.316 sec)
INFO:tensorflow:global_step/sec: 333.087
INFO:tensorflow:loss = 55.027245, step = 7601 (0.293 sec)
INFO:tensorflow:global_step/sec: 306.343
INFO:tensorflow:loss = 50.44211, step = 7701 (0.328 sec)
INFO:tensorflow:global_step/sec: 292.769
INFO:tensorflow:loss = 56.44274, step = 7801 (0.341 sec)
INFO:tensorflow:global_step/sec: 299.219
INFO:tensorflow:loss = 50.994373, step = 7901 (0.335 sec)
INFO:tensorflow:global_step/sec: 315.71
INFO:tensorflow:loss = 74.778046, step = 8001 (0.318 sec)
INFO:tensorflow:global_step/sec: 358.771
INFO:tensorflow:loss = 60.49221, step = 8101 (0.278 sec)
INFO:tensorflow:global_step/sec: 319.185
INFO:tensorflow:loss = 61.860382, step = 8201 (0.312 sec)
INFO:tensorflow:global_step/sec: 347.108
INFO:tensorflow:loss = 63.79139, step = 8301 (0.292 sec)
INFO:tensorflow:global_step/sec: 330.224
INFO:tensorflow:

In [70]:
# Get all weights
for i,name in enumerate(model_LR.get_variable_names()):
    print(name,": ", model_LR.get_variable_value(name))

global_step :  10407
linear/linear_model/Age/weights :  [[-0.01193987]]
linear/linear_model/Age/weights/part_0/Ftrl :  [[6.093922e+08]]
linear/linear_model/Age/weights/part_0/Ftrl_1 :  [[1473.7301]]
linear/linear_model/Cabin/weights :  [[0.00034001]]
linear/linear_model/Cabin/weights/part_0/Ftrl :  [[1.9154797e+10]]
linear/linear_model/Cabin/weights/part_0/Ftrl_1 :  [[-235.29172]]
linear/linear_model/Embarked/weights :  [[-0.05420135]]
linear/linear_model/Embarked/weights/part_0/Ftrl :  [[1760927.6]]
linear/linear_model/Embarked/weights/part_0/Ftrl_1 :  [[359.6258]]
linear/linear_model/Fare/weights :  [[0.01205712]]
linear/linear_model/Fare/weights/part_0/Ftrl :  [[1.04833504e+09]]
linear/linear_model/Fare/weights/part_0/Ftrl_1 :  [[-1951.928]]
linear/linear_model/Honor/weights :  [[1.9490682]]
linear/linear_model/Honor/weights/part_0/Ftrl :  [[15344.565]]
linear/linear_model/Honor/weights/part_0/Ftrl_1 :  [[-1207.1864]]
linear/linear_model/Parch/weights :  [[-0.2595531]]
linear/linear

<h3> Validate the model </h3>

In [71]:
def print_rmse(model, name, X, y):
  metrics = model.evaluate(input_fn = make_input_fn(X, y, 1))
  print('RMSE on {} dataset = {}'.format(name, np.sqrt(metrics['average_loss'])))

In [72]:
print_rmse(model_LR, 'validation', X_valid, y_valid)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-02-15:37:50
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Titanic_trained_LC/model.ckpt-10407
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-02-15:37:51
INFO:tensorflow:Saving dict for global step 10407: accuracy = 0.8206278, accuracy_baseline = 0.5650224, auc = 0.8963754, auc_precision_recall = 0.88440686, average_loss = 0.40983117, global_step = 10407, label/mean = 0.4349776, loss = 45.696175, prediction/mean = 0.40405366
RMSE on validation dataset = 0.6401805877685547


<h3> Prediction X_test </h3>

In [73]:
preds_iter = model_LR.predict(input_fn = make_prediction_input_fn(X_test,1))
# print([pred['predictions'][0] for pred in list(itertools.islice(preds_iter, 5))])
predicted_classes = [np.int(p["classes"]) for p in preds_iter]
print("New Samples, Class Predictions:    {}\n".format(predicted_classes))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Titanic_trained_LC/model.ckpt-10407
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
New Samples, Class Predictions:    [0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 

In [74]:
passengerid = np.array(df_test.PassengerId.tolist())
submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': predicted_classes}).set_index('PassengerId')
submission.to_csv('submission_tf_LR_v4.csv')

In [ ]:
# feed_dict_train = {X_place: X_train}
#         prediction = sess.run(y_pred, feed_dict=feed_dict_train)
#         correct = np.equal(np.argmax(prediction,axis=1), np.argmax(y_train,axis=1))
#         # evaluate the accuracy
#         accuracy = np.mean(correct)
#         print('Accuracy on validation: {:2.2%}'.format (accuracy))
#     save_path = saver.save(sess, model_path+'Titanic_model.ckpt')

<h3> Deep Neural Network regression </h3>

In [75]:
OUTDIR_DNN = 'Titanic_trained_DNN'
tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(OUTDIR_DNN, ignore_errors = True) # start fresh each time
model_DNN = tf.estimator.(hidden_units = [32, 8, 2],
      feature_columns = make_feature_cols(X_train), model_dir = OUTDIR_DNN)
model_DNN.train(input_fn = make_input_fn(X_train, y_train, num_epochs = 2000))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'Titanic_trained_DNN', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1241aa5c0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into Titanic_trained_DNN/model.ckpt.
INFO:tensorflow:loss = 1660.9915, step = 1
INFO:tens

INFO:tensorflow:global_step/sec: 348.124
INFO:tensorflow:loss = 82.48874, step = 7501 (0.287 sec)
INFO:tensorflow:global_step/sec: 322.978
INFO:tensorflow:loss = 85.81544, step = 7601 (0.312 sec)
INFO:tensorflow:global_step/sec: 273.079
INFO:tensorflow:loss = 86.92778, step = 7701 (0.363 sec)
INFO:tensorflow:global_step/sec: 274.93
INFO:tensorflow:loss = 90.81992, step = 7801 (0.364 sec)
INFO:tensorflow:global_step/sec: 330.208
INFO:tensorflow:loss = 84.15419, step = 7901 (0.303 sec)
INFO:tensorflow:global_step/sec: 316.661
INFO:tensorflow:loss = 79.73399, step = 8001 (0.316 sec)
INFO:tensorflow:global_step/sec: 357.85
INFO:tensorflow:loss = 81.38554, step = 8101 (0.279 sec)
INFO:tensorflow:global_step/sec: 336.622
INFO:tensorflow:loss = 85.26166, step = 8201 (0.299 sec)
INFO:tensorflow:global_step/sec: 318.415
INFO:tensorflow:loss = 82.48974, step = 8301 (0.313 sec)
INFO:tensorflow:global_step/sec: 360.723
INFO:tensorflow:loss = 85.818245, step = 8401 (0.278 sec)
INFO:tensorflow:globa

In [76]:
print_rmse(model_DNN, 'validation', X_valid, y_valid)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-02-15:41:21
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Titanic_trained_DNN/model.ckpt-10407
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-02-15:41:21
INFO:tensorflow:Saving dict for global step 10407: accuracy = 0.5650224, accuracy_baseline = 0.5650224, auc = 0.5, auc_precision_recall = 0.71748877, average_loss = 0.694991, global_step = 10407, label/mean = 0.4349776, loss = 77.49149, prediction/mean = 0.36499456
RMSE on validation dataset = 0.8336611986160278


In [77]:
preds_iter = model_DNN.predict(input_fn = make_prediction_input_fn(X_test,1))
# print([pred['predictions'][0] for pred in list(itertools.islice(preds_iter, 5))])
predicted_classes = [np.int(p["classes"]) for p in preds_iter]
print("New Samples, Class Predictions:    {}\n".format(predicted_classes))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Titanic_trained_DNN/model.ckpt-10407
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
New Samples, Class Predictions:    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [78]:
passengerid = np.array(df_test.PassengerId.tolist())
submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': predicted_classes}).set_index('PassengerId')
submission.to_csv('submission_tf_DNN_v4.csv')

<h3> XGBoost </h3>

In [82]:
OUTDIR_DNN = 'Titanic_trained_XGBoost'
tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(OUTDIR_DNN, ignore_errors = True) # start fresh each time
model_XGBoost = XGBClassifier()
model_XGBoost.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [85]:
# print_rmse(model_DNN, 'validation', X_valid, y_valid)
y_pred = model_XGBoost.predict(X_valid)
accuracy = accuracy_score(y_valid, y_pred)
print('accuracy: ', accuracy)

accuracy:  0.8475336322869955


/Users/lambert/virtualenvs/tensorflow/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [86]:
# preds_iter = model_DNN.predict(input_fn = make_prediction_input_fn(X_test,1))
# print([pred['predictions'][0] for pred in list(itertools.islice(preds_iter, 5))])
predicted_classes = model_XGBoost.predict(X_test)
# predicted_classes = [np.int(p["classes"]) for p in preds_iter]
print("New Samples, Class Predictions:    {}\n".format(predicted_classes))

New Samples, Class Predictions:    [0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0 0 0 1 0 0 1 0 1 0 1 0 1 0 0 0 0 0 1 0 0
 0 0 1 0 1 0 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 0 1
 1 0 0 1 0 1 1 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0
 0 0 1 0 0 0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1 1 0 0 1 0 0 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 1 1 0 1 1 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 0 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0
 0 1 1 1 1 0 0 1 0 0 1]



/Users/lambert/virtualenvs/tensorflow/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [87]:
passengerid = np.array(df_test.PassengerId.tolist())
submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': predicted_classes}).set_index('PassengerId')
submission.to_csv('submission_tf_XGBoost_v4.csv')

In [ ]:
# Tensorflow variables
# with tf.device('/device:GPU:0'):
features= len(sFeatures)
X_place = tf.placeholder(tf.float32, shape=[None,features])
y_place = tf.placeholder(tf.float32, shape=[None,2])
W = tf.Variable(tf.random_normal([features, 2]), name='weights')
b = tf.Variable(tf.zeros(2), name='bias')
logits = tf.matmul(X_place, W) + b
y_pred = tf.nn.softmax(logits)

In [ ]:
# Cross Entropy
# with tf.device('/device:GPU:0'):
learning_rate = 0.001
cross_entropy = -tf.reduce_sum(y_place * tf.log(y_pred + 1e-10), axis=1)
# cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_train)
cost = tf.reduce_mean(cross_entropy)
train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
saver = tf.train.Saver()

In [ ]:
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())

In [ ]:
# # training starting option 1
# for epoch in range(10000):
#     total_loss = 0.
#     x_batch, y_batch = next_batch(batch_size, X_train, y_train)
#     feed_dict_train = {X_place: x_batch, y_place: y_batch}
#     _, loss = sess.run([train_op, cost], feed_dict=feed_dict_train)
#     total_loss += loss
#     print('Epoch: {:04d}, total loss={:.9f}'.format(epoch+1, total_loss))
# #     print('Epoch: {:04d}, total loss={:.9f}, entropy: {}'.format(epoch+1, total_loss, cross_entropy))
# #     print('predict: {:.6f}'.format(y_pred))
# print ('Training complete!')

In [ ]:
# sess.close()

In [ ]:
# training starting option 1 with CPU
# for epoch in range(100):
#     total_loss = 0.
#     for i in range(len(X_train)):
#         feed = {X_place: X_train, y_place: y_train}
#         _, loss = sess.run([train_op, cost], feed_dict=feed)
#         total_loss += loss
#     print('Epoch: {:04d}, total loss={:.9f}'.format(epoch+1, total_loss))
# #     print('Epoch: {:04d}, total loss={:.9f}, entropy: {}'.format(epoch+1, total_loss, cross_entropy))
# #     print('predict: {:.6f}'.format(y_pred))
# print ('Training complete!')

In [ ]:
# training starting option 1 with GPU
# with tf.device('/device:GPU:0'):
start = time.time()

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    with tf.device('/device:GPU:0'):
        sess.run(tf.global_variables_initializer())
        for epoch in range(10000):
            total_loss = 0.
            for i in range(len(X_train)):
                feed = {X_place: X_train, y_place: y_train}
                _, loss = sess.run([train_op, cost], feed_dict=feed)
                total_loss += loss
            print('Epoch: {:04d}, total loss={:.9f}'.format(epoch+1, total_loss))
        #     print('Epoch: {:04d}, total loss={:.9f}, entropy: {}'.format(epoch+1, total_loss, cross_entropy))
        #     print('predict: {:.6f}'.format(y_pred))
        print ('Training complete!')

        feed_dict_train = {X_place: X_train}
        prediction = sess.run(y_pred, feed_dict=feed_dict_train)
        correct = np.equal(np.argmax(prediction,axis=1), np.argmax(y_train,axis=1))
        # evaluate the accuracy
        accuracy = np.mean(correct)
        print('Accuracy on validation: {:2.2%}'.format (accuracy))
    save_path = saver.save(sess, model_path+'Titanic_model.ckpt')
end = time.time()
print('time spend: {:.4f} sec'.format(end-start))

In [ ]:
# save_path = saver.save(sess, model_path+'Titanic_model.ckpt')
# sess.close()

In [ ]:
# evaluate the training data
with tf.Session() as sess:
    saver.restore(sess, model_path+'Titanic_model.ckpt')
    feed_dict_train = {X_place: X_train}
    prediction = sess.run(y_pred, feed_dict=feed_dict_train)
    correct = np.equal(np.argmax(prediction,axis=1), np.argmax(y_train,axis=1))
# evaluate the accuracy
accuracy = np.mean(correct)
print('Accuracy on validation: {:2.2%}'.format (accuracy))

In [ ]:
# evaluate the validate data
with tf.Session() as sess:
    saver.restore(sess, model_path+'Titanic_model.ckpt')
    feed_dict_valid = {X_place: X_val}
    prediction = sess.run(y_pred, feed_dict=feed_dict_valid)
    correct = np.equal(np.argmax(prediction,axis=1), np.argmax(y_val,axis=1))
# evaluate the accuracy

accuracy = np.mean(correct)
print('Accuracy on validation: {:2.2%}'.format (accuracy))

In [ ]:
# prediction for test data
with tf.Session() as sess:
    saver.restore(sess, model_path+'Titanic_model.ckpt')
    feed_dict_test = {X_place: X_test}
    prediction = sess.run(y_pred, feed_dict=feed_dict_test)
    y_predict = np.argmax(prediction, axis=1)
    y_predict_1 = np.argmin(prediction, axis=1)
    
# evaluate the accuracy

In [ ]:
passengerid = np.array(df_test.PassengerId.tolist())
submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict_1}).set_index('PassengerId')
submission.to_csv('submission_tf_GDO_v3.csv')

In [ ]:
y_predict

In [ ]:
y_predict_1

In [ ]:
# print_accuracy()
# plot_example_errors()
# optimize(num_iterations=10)
# print_accuracy()
# plot_example_errors()
# plot_weights()

In [ ]:
File_Writer=tf.summary.FileWriter(''/Users/lambert/Documents/Python_code/tensorflow_code/Titanic'
                                  ,sess.graph)

In [ ]:
# with tf.device('/device:GPU:0'):
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:

In [ ]:
# allocate as much GPU memory based on runtime allocations
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [ ]:
# allocate 40% memory of GPU
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
session = tf.Session(config=config, ...)

In [ ]:
# Automatically choose existing in case specified one doesn't exit
# allow_soft_placement=True
with tf.device('/device:GPU:2'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
  c = tf.matmul(a, b)
# Creates a session with allow_soft_placement and log_device_placement set
# to True.
sess = tf.Session(config=tf.ConfigProto(
      allow_soft_placement=True, log_device_placement=True))

In [ ]:
sess.close()

In [ ]:
sys.argv[2]

In [ ]:
device_lib.list_local_devices()

In [ ]:
np.random.random(10)

In [80]:
import xgboost

In [81]:
from xgboost import XGBClassifier